#### Dependencies

In [1]:
'''! pip install nltk
! pip install pandas
! pip install textblob
! pip install matplotlib
! pip install seaborn
! pip install spacy
! pip install transformers'''

'! pip install nltk\n! pip install pandas\n! pip install textblob\n! pip install matplotlib\n! pip install seaborn\n! pip install spacy\n! pip install transformers'

In [1]:
import pandas as pd
import json
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
import json
import re
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from collections import Counter
import string
import spacy
import transformers
from transformers import pipeline

/Users/mafaldaaires/miniconda3/envs/CAC_Proj/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


#### Load Dataset

In [2]:
# Baixar recursos do NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mafaldaaires/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mafaldaaires/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mafaldaaires/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/mafaldaaires/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
def load_yelp_data(file_path, nrows=None):
    data = []
    print(f" Loading: {file_path}...")  

    with open(file_path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f):
            if nrows and i >= nrows:
                break
            data.append(json.loads(line))
            
            if (i + 1) % 100000 == 0:
                print(f"  🔄 {i + 1} loaded lines...")

    print(f" {file_path} loaded with {len(data)} lines!\n")  
    return pd.DataFrame(data)

print("\n Starting to load datasets...\n")


 Starting to load datasets...



In [4]:
df_business = load_yelp_data("yelp_dataset/yelp_academic_dataset_business.json")  # (~150K)
df_review = load_yelp_data("sparks_reviews.json")  # 500K
df_user = load_yelp_data("yelp_dataset/yelp_academic_dataset_user.json")  # 500k
#df_checkin = pd.read_json('data/yelp_academic_dataset_checkin.json', lines=True) not used for now
#df_tip = pd.read_json('data/yelp_academic_dataset_tip.json', lines=True) not used for now

 Loading: yelp_dataset/yelp_academic_dataset_business.json...
  🔄 100000 loaded lines...
 yelp_dataset/yelp_academic_dataset_business.json loaded with 150346 lines!

 Loading: sparks_reviews.json...
 sparks_reviews.json loaded with 40186 lines!

 Loading: yelp_dataset/yelp_academic_dataset_user.json...
  🔄 100000 loaded lines...
  🔄 200000 loaded lines...
  🔄 300000 loaded lines...
  🔄 400000 loaded lines...
  🔄 500000 loaded lines...
  🔄 600000 loaded lines...
  🔄 700000 loaded lines...
  🔄 800000 loaded lines...
  🔄 900000 loaded lines...
  🔄 1000000 loaded lines...
  🔄 1100000 loaded lines...
  🔄 1200000 loaded lines...
  🔄 1300000 loaded lines...
  🔄 1400000 loaded lines...
  🔄 1500000 loaded lines...
  🔄 1600000 loaded lines...
  🔄 1700000 loaded lines...
  🔄 1800000 loaded lines...
  🔄 1900000 loaded lines...
 yelp_dataset/yelp_academic_dataset_user.json loaded with 1987897 lines!



#### **Business Dataset**

In [6]:
df_business

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,AB,T6J 5H2,53.468419,-113.492054,3.0,13,1,"{'ByAppointmentOnly': 'False', 'RestaurantsPri...","Nail Salons, Beauty & Spas","{'Monday': '10:0-19:30', 'Tuesday': '10:0-19:3..."
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,TN,37204,36.115118,-86.766925,4.0,5,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Pets, Nurseries & Gardening, Pet Stores, Hobby...","{'Monday': '9:30-17:30', 'Tuesday': '9:30-17:3..."
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46",Indianapolis,IN,46250,39.908707,-86.065088,3.5,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Shopping, Jewelry, Piercing, Toy Stores, Beaut...",None
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,IL,62025,38.782351,-89.950558,4.0,24,1,"{'BusinessParking': '{'garage': False, 'street...","Fitness/Exercise Equipment, Eyewear & Optician...","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ..."


In [7]:
# See how many unique business_ids we have
total_business_ids_unicos = df_business['business_id'].nunique()
print(f"How many business do we have? {total_business_ids_unicos}")

How many business do we have? 150346


We are going to select only one city of the dataframe.

In [8]:
# Group by 'city' and sum the 'review_count'
city_reviews = df_business.groupby('city')['review_count'].sum()

# Sort in descending order and get the top cities (e.g., top 10)
top_cities = city_reviews.sort_values(ascending=False).head(10)

print("Cities with the highest total review counts:")
print(top_cities)

Cities with the highest total review counts:
city
Philadelphia     936240
New Orleans      621361
Nashville        441053
Tampa            439506
Tucson           387254
Indianapolis     349228
Reno             334610
Santa Barbara    262853
Saint Louis      244360
Boise            101893
Name: review_count, dtype: int64


In [12]:
# Dataframe only with Sparks reviews
df_business_sparks = df_business[df_business['city'] == 'Sparks']

# categories related to restaurants
df_business_sparks_rest = df_business_sparks[
    (df_business_sparks['city'].str.lower() == 'sparks') &
    (df_business_sparks['categories'].str.contains('Restaurants', na=False))
]

In [15]:
city_business_ids = set(df_business_sparks_rest['business_id'])
filtered_reviews = []

with open('yelp_dataset/yelp_academic_dataset_review.json', 'r', encoding='utf-8') as f:
    for line in f:
        review = json.loads(line)
        if review['business_id'] in city_business_ids:
            filtered_reviews.append(review)

restaurant_reviews = pd.DataFrame(filtered_reviews)

with open('sparks_business.json', 'w', encoding='utf-8') as out_file:
    for review in filtered_reviews:
        out_file.write(json.dumps(review) + '\n')

In [11]:
# See how many unique business_ids we have
total_business_ids_unicos = df_business_sparks['business_id'].nunique()
print(f"How many business do we have in Sparks? {total_business_ids_unicos}")

How many business do we have in Sparks? 1624


Ver se o dataframe é balanceado.

In [35]:
distinct_stars = df_business_sparks['stars'].unique()

distinct_stars_sorted = sorted(distinct_stars)

print(distinct_stars_sorted)

[np.float64(1.0), np.float64(1.5), np.float64(2.0), np.float64(2.5), np.float64(3.0), np.float64(3.5), np.float64(4.0), np.float64(4.5), np.float64(5.0)]


In [36]:
# Step 1: Count reviews by star rating
star_counts = df_business_sparks['stars'].value_counts().sort_index()

# Step 2: Calculate total number of reviews
total_reviews = star_counts.sum()

# Display counts and percentages
print("Review counts and percentages by star rating:")
for rating in [1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]:
    count = star_counts.get(rating, 0)
    print(f"Stars {rating}: {count} reviews")

Review counts and percentages by star rating:
Stars 1.0: 16 reviews
Stars 1.5: 53 reviews
Stars 2.0: 105 reviews
Stars 2.5: 146 reviews
Stars 3.0: 200 reviews
Stars 3.5: 270 reviews
Stars 4.0: 305 reviews
Stars 4.5: 314 reviews
Stars 5.0: 215 reviews


I'm going to create two labels for my dataset: 0 (**[1.0, 3.5]**) and 1 (**[4.0, 5.0]**).

In [37]:
# Get all the business_id values in the df_business_sparks dataframe
business_ids = df_business_sparks['business_id'].tolist()

#### **Review Dataset**

In [38]:
df_review.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,LkXdUnLG55HUqClm5C-FRQ,Bcwv94u7FgNCpCoJLmSsaA,l4aS1QVy_bGnwUXSJzpbsw,5.0,0,0,1,Best pizza ever... hands down.\n\nLove the pee...,2009-08-04 22:21:20
1,Z_tQ-hruQRU45Rg3XglyJQ,9wymgENHBBOm9Tlx7xHy7A,HZtkTPfSshfRjpQR7x1Raw,2.0,0,0,0,I didn't really like my sandwich I got here. I...,2014-03-14 23:13:32
2,r34W_h6NEMvBIpT2h53GQw,p0tP-O8y7rhJKd1iFirgwQ,t0XZeA0MP0P66QyLQn_NUw,5.0,0,0,0,This is a must get when you come to Reno is Ye...,2016-07-15 07:24:31
3,P6Kbftcn5GwvIo2xQyXFBg,fSR8LJJ0Aa3w8HoaGyXYCA,UX3eq0WsVva-cqlVrzyzFg,5.0,0,0,0,"Yummy food, they will veganize things for you...",2013-12-28 22:22:19
4,wXcsMgO27jDlr_8xuhn-pQ,CqRAt1eogaHCBBtF_shemg,ruFtZKwlJASx5BTk1dh5AQ,5.0,0,0,0,The new Bacon-Bleu cheeseburger is excellent! ...,2010-03-14 05:14:25


In [39]:
df_review.shape

(40186, 9)

In [40]:
# Filter df_review to keep only reviews for those business_ids
df_review_sparks = df_review[df_review['business_id'].isin(business_ids)]

In [41]:
df_review_sparks.shape

(40186, 9)

In [42]:
# Get all the user_ids values in the df_review_sparks dataframe 
user_ids = df_review_sparks['user_id'].tolist()

In [43]:
from textblob import TextBlob

df_review_sparks['sentiment_analysis'] = df_review_sparks['text'].apply(lambda x: TextBlob(x).sentiment.polarity)
print(df_review_sparks[['text', 'sentiment_analysis']].head())

                                                text  sentiment_analysis
0  Best pizza ever... hands down.\n\nLove the pee...            0.347569
1  I didn't really like my sandwich I got here. I...            0.052976
2  This is a must get when you come to Reno is Ye...            0.415278
3  Yummy food,  they will veganize things for you...            0.481250
4  The new Bacon-Bleu cheeseburger is excellent! ...            0.677273


In [44]:
df_review_sparks['text'][1]

"I didn't really like my sandwich I got here. I got the jalapeño cheese bread and it was tasteless and dry. I could tell it was old. Also I got tomatoes and cucumbers which were mush because they were overripe. Usually they're a lot better than this but today wasn't it. The staff is very friendly though."

In [45]:
# Define a function to combine stars and sentiment
def sentiment_enhanced_label(row):
    stars = row['stars']
    polarity = row['sentiment_analysis']  # From TextBlob
    if stars <= 3.5:
        return 0 if polarity <= 0 else 1  # Adjust threshold as needed
    else:
        return 1 if polarity >= 0 else 0

# Apply the function
df_review_sparks['label'] = df_review_sparks.apply(sentiment_enhanced_label, axis=1)
print(df_review_sparks[['stars', 'sentiment_analysis', 'label']].head())

   stars  sentiment_analysis  label
0    5.0            0.347569      1
1    2.0            0.052976      1
2    5.0            0.415278      1
3    5.0            0.481250      1
4    5.0            0.677273      1


In [46]:
df_review_sparks

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,sentiment_analysis,label
0,LkXdUnLG55HUqClm5C-FRQ,Bcwv94u7FgNCpCoJLmSsaA,l4aS1QVy_bGnwUXSJzpbsw,5.0,0,0,1,Best pizza ever... hands down.\n\nLove the pee...,2009-08-04 22:21:20,0.347569,1
1,Z_tQ-hruQRU45Rg3XglyJQ,9wymgENHBBOm9Tlx7xHy7A,HZtkTPfSshfRjpQR7x1Raw,2.0,0,0,0,I didn't really like my sandwich I got here. I...,2014-03-14 23:13:32,0.052976,1
2,r34W_h6NEMvBIpT2h53GQw,p0tP-O8y7rhJKd1iFirgwQ,t0XZeA0MP0P66QyLQn_NUw,5.0,0,0,0,This is a must get when you come to Reno is Ye...,2016-07-15 07:24:31,0.415278,1
3,P6Kbftcn5GwvIo2xQyXFBg,fSR8LJJ0Aa3w8HoaGyXYCA,UX3eq0WsVva-cqlVrzyzFg,5.0,0,0,0,"Yummy food, they will veganize things for you...",2013-12-28 22:22:19,0.481250,1
4,wXcsMgO27jDlr_8xuhn-pQ,CqRAt1eogaHCBBtF_shemg,ruFtZKwlJASx5BTk1dh5AQ,5.0,0,0,0,The new Bacon-Bleu cheeseburger is excellent! ...,2010-03-14 05:14:25,0.677273,1
...,...,...,...,...,...,...,...,...,...,...,...
40181,4hU3oMaMZX4KnUEzOw3WQg,bVTkQjIcrMVn6XYHljPJKQ,cTpu1MpVItWZCK_ylUThtA,1.0,0,2,0,There are so many issues with this place. \n\n...,2021-01-28 05:16:39,0.025397,1
40182,glDpDm9HhL8IMOZlZnKcQw,sGMB4rFamgxbFOKAmiJ0mg,nTEmSvz8_DK-iK3miQFCwA,4.0,7,3,7,So in the last couple of years we have had hor...,2017-12-10 19:41:59,0.119638,1
40183,oDpz2nfU1J1nzaDcYtEGSw,kkaWSsIDLsKzHOl1UpD2tg,nTEmSvz8_DK-iK3miQFCwA,3.0,2,0,1,We were going to a different restaurant this m...,2017-02-18 21:27:15,0.294444,1
40184,La8GFrkoNP72qHjXdwqeKA,2h7J39cjxmlwR0DCUKQ_2Q,nTEmSvz8_DK-iK3miQFCwA,5.0,0,0,0,Fantastic service! Super crowded for a mining ...,2021-12-09 05:40:02,0.298658,1


### Business_id

In [47]:
# business_id can have several reviews => we have to calculate the average of the labels given to the business as the final label

business_labels = df_review_sparks.groupby('business_id')['label'].mean().reset_index()
business_labels.columns = ['business_id', 'avg_label']

In [48]:
df_business_sparks = df_business_sparks.merge(business_labels, on='business_id', how='left')

In [49]:
df_business_sparks

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,avg_label
0,fSCNwMtNNQY9QT69Cj9fiA,Sierra Pro Events,,Sparks,NV,89431,39.540154,-119.748395,5.0,7,1,"{'BusinessAcceptsCreditCards': 'True', 'WiFi':...","Musicians, DJs, Karaoke, Event Planning & Serv...","{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ...",NaN
1,Lx8ZYUJ_-3Aa8N821mN1WQ,ALM Kainan,"390 N McCarran Blvd, Ste B",Sparks,NV,89434,39.537179,-119.735532,4.0,13,0,"{'RestaurantsAttire': 'u'casual'', 'HasTV': 'F...","Restaurants, Filipino","{'Monday': '10:0-19:0', 'Tuesday': '10:0-19:0'...",0.846154
2,UX3eq0WsVva-cqlVrzyzFg,Cantina Los Tres Hombres,926 Victorian Ave,Sparks,NV,89431,39.535111,-119.754857,3.0,284,1,"{'HasTV': 'True', 'RestaurantsPriceRange2': '2...","Bars, Nightlife, Mexican, Restaurants, Seafood","{'Monday': '11:0-20:0', 'Tuesday': '11:0-20:0'...",0.787671
3,3vTG6LQyr3obzfvrK1l1hA,7-Eleven,15 Eagle Canyon Dr,Sparks,NV,89441,39.638503,-119.712869,2.0,12,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Food, Gas Stations, Service Stations, Convenie...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",NaN
4,YhQAQhG1IkXJH7oLZMbM9w,Del Taco,252 Los Altos Pkwy,Sparks,NV,89431,39.583653,-119.740935,1.5,60,1,"{'RestaurantsGoodForGroups': 'True', 'GoodForK...","Restaurants, Fast Food, Mexican","{'Monday': '8:0-23:0', 'Tuesday': '8:0-23:0', ...",0.349206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1619,km3eDGqOMxygC3UviNxseg,PMI Nevada,"979 Pyramid Way, Ste 115",Sparks,NV,89431,39.542812,-119.752994,1.5,7,1,None,"Home Services, Real Estate, Property Management","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",NaN
1620,lna-UaP3ZH0EjFgYNSOQZw,Marina's Edge,550 Howard Dr Ofc,Sparks,NV,89434,39.554898,-119.731043,1.5,16,1,"{'ByAppointmentOnly': 'False', 'BusinessAccept...","Apartments, Home Services, Real Estate","{'Monday': '8:30-17:30', 'Tuesday': '8:30-17:3...",NaN
1621,v2M2ENlB1cqgWd7-IWzgbw,Enterprise Rent-A-Car,"1450 E Prater Way, Ste 113",Sparks,NV,89434,39.543472,-119.717377,4.0,17,1,None,"Hotels & Travel, Car Rental","{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:30', ...",NaN
1622,nTEmSvz8_DK-iK3miQFCwA,Great Basin Brewing,846 Victorian Ave,Sparks,NV,89431,39.535095,-119.754133,4.0,580,1,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...","Food, Bars, Restaurants, Brewpubs, Breweries, ...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ...",0.948161


### Categories Analysis

In [68]:
# Get all distinct categories from df_business_sparks

# Check if the column exists first
if 'Categories' not in df_business_sparks.columns and 'categories' in df_business_sparks.columns:
    # Handle case sensitivity issue
    categories_col = 'categories'
    print("Note: Using 'categories' column (lowercase)")
else:
    categories_col = 'Categories'

# Method 1: If categories are stored as single values per row
unique_categories = df_business_sparks[categories_col].dropna().unique()
print(f"Number of unique category entries: {len(unique_categories)}")

# Method 2: If categories are stored as comma-separated lists
# This will split categories and extract all unique values
all_categories = set()

# Skip rows with NaN values
for categories_str in df_business_sparks[categories_col].dropna():
    # Check if it's a string type (could be a list or other format)
    if isinstance(categories_str, str):
        # Split by comma and strip whitespace
        category_list = [cat.strip() for cat in categories_str.split(',')]
        all_categories.update(category_list)
    elif isinstance(categories_str, list):
        # If already a list, add all items
        all_categories.update(categories_str)

print(f"\nTotal number of distinct categories: {len(all_categories)}")
print("\nList of all distinct categories:")
for category in sorted(all_categories):
    print(f"- {category}")

# If there are too many categories to display comfortably, show just a sample
if len(all_categories) > 50:
    print("\nNote: There are many categories. Here's a sample of 20:")
    import random
    sample_categories = random.sample(list(all_categories), min(20, len(all_categories)))
    for category in sorted(sample_categories):
        print(f"- {category}")
    
    # Offer to save to file
    print("\nTo see all categories, you can save them to a file using:")
    print("pd.Series(list(all_categories)).to_csv('all_categories.csv', index=False)")

Note: Using 'categories' column (lowercase)
Number of unique category entries: 284

Total number of distinct categories: 136

List of all distinct categories:
- Active Life
- Adult Entertainment
- American (New)
- American (Traditional)
- Arts & Entertainment
- Asian Fusion
- Bagels
- Bakeries
- Barbeque
- Bars
- Basque
- Batting Cages
- Beer
- Beer Bar
- Bowling
- Breakfast & Brunch
- Breweries
- Brewpubs
- Bubble Tea
- Buffets
- Burgers
- Butcher
- Cafes
- Cajun/Creole
- Cantonese
- Casinos
- Caterers
- Cheesesteaks
- Chicken Shop
- Chicken Wings
- Chinese
- Cocktail Bars
- Coffee & Tea
- Coffee Roasteries
- Comfort Food
- Country Clubs
- Country Dance Halls
- Creperies
- Cupcakes
- Delis
- Desserts
- Dim Sum
- Diners
- Dive Bars
- Donuts
- Drugstores
- Ethnic Food
- Ethnic Grocery
- Event Planning & Services
- Fast Food
- Filipino
- Fish & Chips
- Food
- Food Court
- Food Delivery Services
- Food Stands
- Food Trucks
- Gastropubs
- Gelato
- Gluten-Free
- Golf
- Greek
- Grocery
- Hal

In [ ]:
city_restaurants = df_business[
    (df_business['city'].str.lower() == 'sparks') &
    (df_business['categories'].str.contains('Restaurants', na=False))
]

In [ ]:
'''
city_business_ids = set(city_restaurants['business_id'])
filtered_reviews = []

with open('data/yelp_academic_dataset_review.json', 'r', encoding='utf-8') as f:
    for line in f:
        review = json.loads(line)
        if review['business_id'] in city_business_ids:
            filtered_reviews.append(review)

restaurant_reviews = pd.DataFrame(filtered_reviews)

with open('data/sparks_reviews.json', 'w', encoding='utf-8') as out_file:
    for review in filtered_reviews:
        out_file.write(json.dumps(review) + '\n')
'''

### PROBLEM

In [50]:
exists = df_business_sparks['avg_label'].isna().any()
print(f"Does NaN exist in df_business_sparks['avg_label']? {exists}")

Does NaN exist in df_business_sparks['avg_label']? True


In [62]:
# business_id == "WnT9NIzQgLlILjPT0kEcsQ" should have an avg_label, however there isn't any review in df_review_sparks with business_id == "WnT9NIzQgLlILjPT0kEcsQ"
# This is not the only case, there are other business_id values that have the same issue
# Should we remove the rows with these business_id values from df_business_sparks (???)

print(df_review_sparks['business_id'].any() == "WnT9NIzQgLlILjPT0kEcsQ")

False


In [64]:
# Step 1: Remove rows where avg_label is NaN
df_business_sparks = df_business_sparks.dropna(subset=['avg_label']).copy()

df_business_sparks

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,avg_label
1,Lx8ZYUJ_-3Aa8N821mN1WQ,ALM Kainan,"390 N McCarran Blvd, Ste B",Sparks,NV,89434,39.537179,-119.735532,4.0,13,0,"{'RestaurantsAttire': 'u'casual'', 'HasTV': 'F...","Restaurants, Filipino","{'Monday': '10:0-19:0', 'Tuesday': '10:0-19:0'...",0.846154
2,UX3eq0WsVva-cqlVrzyzFg,Cantina Los Tres Hombres,926 Victorian Ave,Sparks,NV,89431,39.535111,-119.754857,3.0,284,1,"{'HasTV': 'True', 'RestaurantsPriceRange2': '2...","Bars, Nightlife, Mexican, Restaurants, Seafood","{'Monday': '11:0-20:0', 'Tuesday': '11:0-20:0'...",0.787671
4,YhQAQhG1IkXJH7oLZMbM9w,Del Taco,252 Los Altos Pkwy,Sparks,NV,89431,39.583653,-119.740935,1.5,60,1,"{'RestaurantsGoodForGroups': 'True', 'GoodForK...","Restaurants, Fast Food, Mexican","{'Monday': '8:0-23:0', 'Tuesday': '8:0-23:0', ...",0.349206
5,qbPsfIKJZQAlrPaYmSyZuQ,All Wrapped Up,,Sparks,NV,89431,39.540154,-119.748395,4.0,7,1,"{'RestaurantsDelivery': 'False', 'GoodForMeal'...","Food Trucks, Food, Salad, Sandwiches, Caterers...",None,0.750000
8,V4THIJk9uhoxL7f06fK5VA,Roundabout Catering,631 Dunn Cir,Sparks,NV,89431,39.528286,-119.734864,5.0,38,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Caterers, Party Equipment Rentals, Food, Venue...","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",0.974359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1599,lXCa6VyCtBSASrsq0QqooQ,Aguitas Bar & Grill Night Club,1825 Prater Way,Sparks,NV,89431,39.537045,-119.767731,4.5,21,1,"{'GoodForKids': 'True', 'RestaurantsDelivery':...","Mexican, Bars, Restaurants, Karaoke, Nightlife","{'Tuesday': '10:0-0:0', 'Wednesday': '10:0-2:0...",1.000000
1600,dLbQ8vOVpAJNjYLM1a3yJA,Oriental Express,"970 S McCarran Blvd, Ste 102",Sparks,NV,89431,39.523694,-119.739791,3.0,11,0,"{'GoodForKids': 'True', 'RestaurantsAttire': '...","Chinese, Restaurants","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",0.833333
1609,RKDnuTw1lJw1Ja-0Xx8aXw,Mother of Macros Meal Prep,1127 Gator Way,Sparks,NV,89431,39.529160,-119.756990,5.0,21,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Food Delivery Services, Food, Specialty Food, ...",None,0.952381
1613,a69pvnfQKMrQ4WXIMdMLCg,Jimmy John's,"475 Sparks Blvd, Ste. 121",Sparks,NV,89434,39.535404,-119.716089,2.5,78,1,"{'BusinessAcceptsCreditCards': 'True', 'Alcoho...","Restaurants, Delis, Fast Food, Sandwiches","{'Monday': '10:30-20:0', 'Tuesday': '10:30-20:...",0.765432


In [65]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

# Create a feature matrix: sentiment label, stars, and review count
features = df_business_sparks[['avg_label', 'stars', 'review_count']].fillna(0).values

# Normalize features
features = (features - features.mean(axis=0)) / features.std(axis=0)

# Compute similarity between businesses
similarity_matrix = cosine_similarity(features)

# Improved function to recommend top N businesses
def recommend_businesses(business_id, top_n=5):
    # Check if business exists in the DataFrame
    business_idx = df_business_sparks.index[df_business_sparks['business_id'] == business_id].tolist()
    
    if not business_idx:
        print(f"Business ID '{business_id}' not found in dataset.")
        return pd.DataFrame(columns=['business_id', 'name', 'avg_label', 'stars'])
    
    idx = business_idx[0]
    sim_scores = list(enumerate(similarity_matrix[idx]))
    
    # Sort by similarity score (descending) and exclude the query business itself
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    
    # If there are no similar businesses
    if not sim_scores:
        print(f"No similar businesses found for '{business_id}'.")
        return pd.DataFrame(columns=['business_id', 'name', 'avg_label', 'stars'])
    
    # Get indices of recommended businesses
    business_indices = [i[0] for i in sim_scores]
    
    # Return recommended businesses with similarity scores
    recommendations = df_business_sparks.iloc[business_indices].copy()
    recommendations['similarity_score'] = [score for _, score in sim_scores]
    
    return recommendations[['business_id', 'name', 'avg_label', 'stars', 'similarity_score']]

# Example usage:


# First check if the business ID exists
if 'MTSW4McQd7CbVtyjqoe9mw' in df_business_sparks['business_id'].values:
    print(recommend_businesses('MTSW4McQd7CbVtyjqoe9mw'))
else:
    print("Let's try with a business ID that exists in the dataset:")
    # Use the first business ID in the dataset as an example
    sample_id = df_business_sparks['business_id'].iloc[0]
    print(f"Using sample business ID: {sample_id}")
    print(recommend_businesses(sample_id))

Let's try with a business ID that exists in the dataset:
Using sample business ID: Lx8ZYUJ_-3Aa8N821mN1WQ
                 business_id                         name  avg_label  stars  \
244   9W06GSzBeIhJyeImD_aNhA                   O'Cleary's   0.787986    3.0   
1104  tY3Y5NzBBD67uwZvBmbncw                   Oyster Bar   0.778146    3.0   
1342  fE8Vyo9HwsQIE9z3KuoWUw               Campo - Sparks   0.776119    3.0   
721   sZhBIAwbTR3FxHMh04gLCQ                  Fuddruckers   0.793722    3.0   
1338  5j9TfW8RrdkpaHlaBwt6bQ  BJ's Restaurant & Brewhouse   0.797317    3.0   

      similarity_score  
244           0.999665  
1104          0.998691  
1342          0.990123  
721           0.970342  
1338          0.960875  


#### **User Dataset**

In [66]:
df_user.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,...,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,"2009,2010,2011","enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",28,...,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,...,1,0,0,0,1,1,0,0,0,0


In [67]:
df_user.shape

(1987897, 22)

In [57]:
# Filter df_user to keep only those user_ids
df_user_sparks = df_user[df_user['user_id'].isin(user_ids)]

df_user_sparks.shape

(20175, 22)

### User_id

In [58]:
user_labels = df_review_sparks.groupby('user_id')['label'].mean().reset_index()
user_labels.columns = ['user_id', 'avg_label']

In [59]:
df_user_sparks = df_user_sparks.merge(user_labels, on='user_id', how='left')

In [60]:
# Doesn't have NaN values
exists = df_user_sparks['avg_label'].isna().any()
print(f"Does NaN exist in df_user_sparks['avg_label']? {exists}")

Does NaN exist in df_user_sparks['avg_label']? False


In [61]:
# Recomendation based on user_id